Importing Necessaey Libraries & GPU Enabling

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data_dir = "./data/chest_xray"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")


Using device: cuda


Data Preprocessing

In [ ]:
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ]),
}


Data Loading

In [ ]:
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms["train"])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms["test"])

batch_size = 128
data_loaders = {
    "train": DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
    "test": DataLoader(test_dataset, batch_size=batch_size, shuffle=False),
}

print(f"Training dataset size: {len(train_dataset)}")
print(f"Testing dataset size: {len(test_dataset)}")

Training dataset size: 5216
Testing dataset size: 624


CNN Model Building

In [ ]:
class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout2d(p=0.4),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x



In [ ]:
# Instantiating Our Constructed Model
model = PneumoniaCNN().to(device)

# Defining loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Setting training loop
num_epochs = 10
train_loss = []
val_loss = []


In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in data_loaders["train"]:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)     # Forward pass happens here
        loss = criterion(outputs, labels)     # Compute loss using Our loss function
        loss.backward()     # Backpropagation happens here
        optimizer.step()     # Update our model weights
        running_loss += loss.item() * inputs.size(0)

    train_loss.append(running_loss / len(train_dataset))

    # Model Validation
    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in data_loaders["test"]:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)

    val_loss.append(val_running_loss / len(test_dataset))
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss[-1]:.4f}, Validation Loss: {val_loss[-1]:.4f}")

Epoch 1/10, Training Loss: 0.6308, Validation Loss: 2.0150
Epoch 2/10, Training Loss: 0.1515, Validation Loss: 0.8120
Epoch 3/10, Training Loss: 0.1069, Validation Loss: 0.7480
Epoch 4/10, Training Loss: 0.0835, Validation Loss: 0.9755
Epoch 5/10, Training Loss: 0.0697, Validation Loss: 1.1536
Epoch 6/10, Training Loss: 0.0606, Validation Loss: 0.9068
Epoch 7/10, Training Loss: 0.0465, Validation Loss: 1.0038
Epoch 8/10, Training Loss: 0.0392, Validation Loss: 1.0232
Epoch 9/10, Training Loss: 0.0321, Validation Loss: 0.9087
Epoch 10/10, Training Loss: 0.0291, Validation Loss: 0.7419


In [ ]:
# Evaluating Our model
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in data_loaders["test"]:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())


In [ ]:
# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=train_dataset.classes))

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.93      0.54      0.68       234
   PNEUMONIA       0.78      0.97      0.87       390

    accuracy                           0.81       624
   macro avg       0.85      0.76      0.77       624
weighted avg       0.83      0.81      0.80       624

Confusion Matrix:
[[126 108]
 [ 10 380]]
